# Scavenger

## Setup

The setup script `scavenger-setup.sh` starts the `minikube` server, creates a namespace `scavenger`, sets the current context to the newly created namespace, and creates the resources defined in the `k8s` directory.

In [1]:
%pycat scavenger-setup.sh

# Run only once minikube and relevant dependencies installed
if [ ! -f /usr/local/bin/minikube ]; then
  echo "Minikube not found. Please follow installation instructions in README.md"
fi
minikube start
kubectl create namespace scavenger
kubectl config set-context --current --namespace=scavenger
cd k8s
IFS=$'\n\t'
for f in $(ls)
do
        until kubectl create -f $f 2> /dev/null
        do
                sleep 10
        done
done


Let us execute the setup script

In [2]:
%%bash
./scavenger-setup.sh

* minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
* Automatically selected the docker driver. Other choices: none, ssh
* Using Docker driver with root privileges
* Starting control plane node minikube in cluster minikube
* Pulling base image ...
* Creating docker container (CPUs=2, Memory=2200MB) ...
* Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
  - Generating certificates and keys ...
  - Booting up control plane ...
  - Configuring RBAC rules ...
* Configuring bridge CNI (Container Networking Interface) ...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
* Verifying Kubernetes components...
* Enabled addons: storage-provisioner, default-storageclass
* Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
namespace/scavenger created
Context "minikube" modified.
deployment.apps/nginx-deployment created
pod/nginx created
pod/postgres created


## Explore

### Mark I: New Beginnings

__0. What are the total number of Pods?__

In [3]:
!kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS              RESTARTS   AGE
kube-system   coredns-5d78c9869d-zjggh           1/1     Running             0          13s
kube-system   etcd-minikube                      1/1     Running             0          25s
kube-system   kube-apiserver-minikube            1/1     Running             0          25s
kube-system   kube-controller-manager-minikube   1/1     Running             0          28s
kube-system   kube-proxy-2tp59                   1/1     Running             0          14s
kube-system   kube-scheduler-minikube            1/1     Running             0          25s
kube-system   storage-provisioner                1/1     Running             0          22s
scavenger     nginx                              0/1     ContainerCreating   0          9s
scavenger     nginx-deployment-55f598f8d-cmthq   0/1     ContainerCreating   0          13s
scavenger     nginx-deployment-55f598f8d-mq2ll   0/1     ContainerCreating   0   

In [4]:
%%bash
NB_PODS_TOTAL=$(kubectl get pods --all-namespaces | tail -n +2 | wc -l)
echo "Number of Total Pods: $NB_PODS_TOTAL"

Number of Total Pods: 12


__1. How many namespaces are there?__

In [5]:
!kubectl get namespaces

NAME              STATUS   AGE
default           Active   36s
kube-node-lease   Active   36s
kube-public       Active   36s
kube-system       Active   36s
scavenger         Active   27s


In [6]:
%%bash
NB_NAMESPACES=$(kubectl get namespaces | tail -n +2 | wc -l)
echo "Number of Namespaces: $NB_NAMESPACES"

Number of Namespaces: 5


__2. How many nodes do we have?__

In [7]:
!kubectl get nodes

NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   41s   v1.27.4


In [8]:
%%bash
NB_NODES=$(kubectl get nodes | tail -n +2 | wc -l)
echo "Number of Nodes: $NB_NODES"

Number of Nodes: 1


__3. How many failing Pods are there?__

In [9]:
!kubectl get pods | grep -Pv '\s+([1-9]+[\d]*)\/\1\s+'

NAME                               READY   STATUS             RESTARTS   AGE
nginx                              0/1     ImagePullBackOff   0          25s
postgres                           0/1     Error              0          24s


In [10]:
%%bash
NB_PODS_FAILING=$(kubectl get pods \
  | grep -Pv '\s+([1-9]+[\d]*)\/\1\s+' \
  | tail -n +2 \
  | wc -l
)
echo "Number of Pods Failing: $NB_PODS_FAILING"

Number of Pods Failing: 2


__4. How many Deployments do we have?__

In [11]:
%%bash
kubectl get deployments -A

NAMESPACE     NAME               READY   UP-TO-DATE   AVAILABLE   AGE
kube-system   coredns            1/1     1            1           49s
scavenger     nginx-deployment   3/3     3            3           44s


In [13]:
%%bash
NB_DEPLOYMENTS=$(kubectl get deployments -A | tail -n +2 | wc -l)
echo $NB_DEPLOYMENTS

2


__5. For the Deployments you discovered, how many ReplicaSets are there in total?__


In [14]:
!kubectl get rs -A

NAMESPACE     NAME                         DESIRED   CURRENT   READY   AGE
kube-system   coredns-5d78c9869d           1         1         1       55s
scavenger     nginx-deployment-55f598f8d   3         3         3       55s


### Mark II: Not-So-New Beginnings

__6. Locate a Postgres Pod within our cluster. Why is it failing? Please provide the variable name(s) that is missing.__

The `postgres` pod was failing because the environment variable `POSTGRESS_PASSWORD` was not set.

In [15]:
!kubectl logs postgres

Error: Database is uninitialized and superuser password is not specified.
       You must specify POSTGRES_PASSWORD to a non-empty value for the
       superuser. For example, "-e POSTGRES_PASSWORD=password" on "docker run".

       You may also use "POSTGRES_HOST_AUTH_METHOD=trust" to allow all
       connections without a password. This is *not* recommended.

       See PostgreSQL documentation about "trust":
       https://www.postgresql.org/docs/current/auth-trust.html


__7. Locate a failed Nginx Pod within our cluster. Why is it failing? Please copy and paste the specific kubectl log you find. Please escape " with \".__

In [16]:
!kubectl logs nginx

Error from server (BadRequest): container "nginx" in pod "nginx" is waiting to start: image can't be pulled


### Mark III: Fixing Our Bugs



__8. For the Postgres Pod, can you fix the ./k8s/postgres-pod.yml file to spin up the image correctly? Please add the environment variable needed, with the value "supersecret". added to your lines of code.__

We can fix the `postgres` pod by adding an environment variable for `POSTGRES_PASSWORD`

In [17]:
%%writefile k8s/postgres-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: postgres
spec:
  containers:
    - name: postgres
      image: postgres
      env:
        - name: POSTGRES_PASSWORD
          value: supersecret

Overwriting k8s/postgres-pod.yml


We can delete the failing pod and apply the corrected config to create a new pod.

In [18]:
%%bash
kubectl delete pod postgres
kubectl apply -f k8s/postgres-pod.yml

pod "postgres" deleted
pod/postgres created


We can confirm that the `postgres` pod is running now!

In [20]:
!kubectl get pods

NAME                               READY   STATUS             RESTARTS   AGE
nginx                              0/1     ImagePullBackOff   0          81s
nginx-deployment-55f598f8d-cmthq   1/1     Running            0          85s
nginx-deployment-55f598f8d-mq2ll   1/1     Running            0          85s
nginx-deployment-55f598f8d-wwrmh   1/1     Running            0          85s
postgres                           1/1     Running            0          5s


__9. For the failed Nginx Pod, can you fix the ./k8s/nignx-pod.yml file to spin up the image correctly? Add the lines of code you added/corrected, and please escape " with \".__

The `nginx` pod is failing because of a typo in the image name (`nginz` instead of `nginx`). We can overwrite the config with the correct image name. 

In [21]:
%%writefile k8s/nginx-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  containers:
    - name: nginx
      image: nginx

Overwriting k8s/nginx-pod.yml


We can delete the pod and apply the correct pod configuration.

In [22]:
%%bash
kubectl delete pod nginx
kubectl apply -f k8s/nginx-pod.yml

pod "nginx" deleted
pod/nginx created


We can confirm that the `nginx` pod is up and running after we fixed the pod config.

In [24]:
!kubectl get pods

NAME                               READY   STATUS    RESTARTS   AGE
nginx                              1/1     Running   0          6s
nginx-deployment-55f598f8d-cmthq   1/1     Running   0          98s
nginx-deployment-55f598f8d-mq2ll   1/1     Running   0          98s
nginx-deployment-55f598f8d-wwrmh   1/1     Running   0          98s
postgres                           1/1     Running   0          18s


## Teardown

Let us teardown the resources we created by deleting all pods, deployments and namespaces, and stopping the minikube server.

In [25]:
%%bash
kubectl delete pods --all
kubectl delete deployments --all
kubectl delete namespace scavenger
minikube stop
minikube delete

pod "nginx" deleted
pod "nginx-deployment-55f598f8d-cmthq" deleted
pod "nginx-deployment-55f598f8d-mq2ll" deleted
pod "nginx-deployment-55f598f8d-wwrmh" deleted
pod "postgres" deleted
deployment.apps "nginx-deployment" deleted
namespace "scavenger" deleted
* Stopping node "minikube"  ...
* Powering off "minikube" via SSH ...
* 1 node stopped.
* Deleting "minikube" in docker ...
* Deleting container "minikube" ...
* Removing /home/codespace/.minikube/machines/minikube ...
* Removed all traces of the "minikube" cluster.
